In [1]:
import matplotlib.pyplot as plt
import pandas as pd

In [11]:
PCSA_df = pd.DataFrame(pd.read_csv("Primary_Care_Shortage_Areas_(PCSA).csv"))
PCSA_df.dropna(inplace=True)
PCSA_df.head() 

,OBJECTID,UNIT_COUNT,CNTY_FIPS,COUNTY,MSSA_ID,MSSA_NAME,DEFINITION,POPDENSITY,AREA_SQMI,POP,PCSA,PCSA_SCORE,PC_PHYS,PC_PHYS_R_CIV,POVScore,PHYSRatioScore,Shape__Area,Shape__Length
0,1,11,1,Alameda,1.1,Livermore Central and West/Spring Town,Urban,1752.000139,0.472102,43860,0,2,48.0,1186.104167,1,1,0.008283,0.489279
1,2,8,1,Alameda,1.2,Altamont/Livermore East/Midway/Mountain House/...,Rural,116.545153,1.950268,32480,1,6,12.0,3167.333333,1,5,0.086298,1.651976
2,3,12,7,Butte,10,Oroville/Palermo/Thermalito,Rural,175.468375,7.768868,42656,1,5,34.0,1408.352941,4,1,0.075224,1.615445
3,4,1,49,Modoc,100,Adin/Lookout/Newell,Frontier,2.315574,956.134376,1454,1,7,0.0,0.000000,2,5,0.267075,3.281548
4,5,2,51,Mono,102,Benton/Bridgeport/Chalfant/Crowley Lake/Colevi...,Frontier,1.957937,1452.267904,5343,1,6,1.0,5951.000000,1,5,0.813804,5.793963


In [12]:
PCSA_df.drop(columns=['OBJECTID', 'MSSA_ID', 'MSSA_NAME', 'COUNTY', 'DEFINITION', 'Shape__Area', 'Shape__Length'], inplace=True)
PCSA_df.head()

,UNIT_COUNT,CNTY_FIPS,POPDENSITY,AREA_SQMI,POP,PCSA,PCSA_SCORE,PC_PHYS,PC_PHYS_R_CIV,POVScore,PHYSRatioScore
0,11,1,1752.000139,0.472102,43860,0,2,48.0,1186.104167,1,1
1,8,1,116.545153,1.950268,32480,1,6,12.0,3167.333333,1,5
2,12,7,175.468375,7.768868,42656,1,5,34.0,1408.352941,4,1
3,1,49,2.315574,956.134376,1454,1,7,0.0,0.000000,2,5
4,2,51,1.957937,1452.267904,5343,1,6,1.0,5951.000000,1,5


In [13]:
PCSA_df = PCSA_df.groupby('CNTY_FIPS').agg({'UNIT_COUNT':'sum', 'POPDENSITY':'median', 'AREA_SQMI':'sum', 'POP':'sum', 
                                            'PCSA':'sum', 'PCSA_SCORE':'median', 'PC_PHYS':'sum', 'PC_PHYS_R_CIV':'median', 
                                            'POVScore':'median', 'PHYSRatioScore':'median'})
PCSA_df.head(10)

,UNIT_COUNT,POPDENSITY,AREA_SQMI,POP,PCSA,PCSA_SCORE,PC_PHYS,PC_PHYS_R_CIV,POVScore,PHYSRatioScore
CNTY_FIPS,,,,,,,,,,
1,360,5829.877145,14.838507,1206808,6,3.375000,2243.000000,1530.064233,2.000000,1.375000
3,1,1.567572,743.187747,1116,1,8.000000,0.000000,0.000000,3.000000,5.000000
5,9,54.689457,347.749367,32435,1,4.333333,37.000000,2544.617647,2.000000,2.333333
7,51,400.046827,944.373893,188610,4,5.375000,209.000000,2018.521572,3.000000,2.375000
9,10,43.539210,102.578333,40020,0,4.000000,26.000000,1700.807692,2.000000,2.000000
11,5,24.308068,1088.981974,9269,3,6.000000,18.000000,2957.343750,2.250000,3.750000
13,207,2299.630641,21.166145,813373,4,3.454545,1215.000000,1776.398966,1.727273,1.727273
15,7,27.979075,805.684044,22835,0,4.000000,29.000000,824.000000,4.000000,0.000000
17,42,235.005239,201.242189,159054,2,4.600000,118.000000,3224.644828,1.800000,2.800000
